# Temporal Factor Autoencoder (TFA) - Demo

**核心创新：用Transformer学习时变因子权重**

本notebook展示TFA的核心功能：
1. 加载和准备数据（36个月序列）
2. 训练TFA模型
3. 可视化动态因子权重
4. 分析学习到的latent factors
5. 生成交易信号


In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from pathlib import Path

from src.data_loader import SequenceDataLoader
from src.models_tfa import TFAPredictor, TemporalFactorAutoencoder
from src.tfa_analysis import TFAAnalyzer
from src.config import Config
from src.utils import set_random_seed, check_gpu_availability

sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', 20)
set_random_seed(42)

print("✅ Libraries loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {check_gpu_availability()}")


## 1. 加载数据

**重要变化**: 序列长度从12个月改为**36个月**


In [ ]:
# 加载配置
config = Config()
device = check_gpu_availability()

# 加载PCA数据（36个月序列）
PCA_PATH = '../feature/pca_feature_store.csv'
data_loader = SequenceDataLoader(
    pca_path=PCA_PATH,
    sequence_length=36,  # ← 改成36个月！
    forward_fill_limit=3
)

# 数据统计
stats = data_loader.get_statistics()
print(f"\\n📊 Dataset Statistics:")
print(f"  Dates: {stats['n_dates']}")
print(f"  Assets: {stats['n_assets']}")
print(f"  PCA Features: {stats['n_features']}")
print(f"  Date Range: {stats['date_range'][0].date()} to {stats['date_range'][1].date()}")
print(f"  Avg Assets/Date: {stats['avg_assets_per_date']:.1f}")

data_loader.df.head()


## 2. 初始化TFA模型

展示模型架构和参数


In [ ]:
# 创建TFA模型
tfa = TFAPredictor(
    n_pca_factors=stats['n_features'],  # 11
    seq_len=36,                         # 3年历史
    d_model=128,                        # 模型维度
    n_heads=8,                          # 8个注意力头
    n_encoder_layers=4,                 # 4层encoder
    n_decoder_layers=2,                 # 2层decoder
    n_latent_factors=5,                 # 学习5个latent factors
    dropout=0.1,
    n_classes=5,                        # 5-分位数分类
    lr=1e-3,
    epochs=20,  # 减少epochs用于快速演示
    batch_size=128,
    alpha=0.1,   # 重构loss权重
    beta=0.05,   # 平滑性loss权重
    gamma=0.01,  # 正交性loss权重
    device=device
)

print("\\n🤖 TFA Model Initialized:")
print(f"  Total Parameters: {tfa.model.count_parameters():,}")
print(f"  Input: (batch, 36 months, 11 PCA factors)")
print(f"  Output: (batch, 5 quantile classes)")
print(f"\\n  Encoder: 4 layers × 8 heads")
print(f"  Decoder: 2 layers × 8 heads")
print(f"  Latent Factors: 5")


## 3. 准备训练数据（小样本演示）

为了快速演示，我们只用一小部分数据


In [ ]:
# 选择一个测试日期
test_dates = data_loader.dates[-20:]  # 最近20个月

# 构建训练集（用前15个月）
X_train_list = []
y_train_list = []

for date in test_dates[:15]:
    try:
        data_dict = data_loader.build_sequences(date, include_target=True, return_dict=True)
        X_train_list.append(data_dict['X'])
        y_train_list.append(data_dict['y'])
    except:
        continue

X_train = np.vstack(X_train_list)
y_train = np.concatenate(y_train_list)

print(f"\\n📦 Training Data:")
print(f"  X_train shape: {X_train.shape}  # (samples, 36 months, 11 features)")
print(f"  y_train shape: {y_train.shape}  # (samples,)")
print(f"  Returns range: [{y_train.min():.3f}, {y_train.max():.3f}]")


## 4. 训练TFA模型

观察多任务loss的变化


In [ ]:
# 训练模型
print("🚀 Training TFA...\\n")

tfa.fit(X_train, y_train, verbose=True)

print("\\n✅ Training completed!")

# 可视化训练历史
if len(tfa.training_history) > 0:
    history_df = pd.DataFrame(tfa.training_history)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    
    # Total loss
    axes[0].plot(history_df['epoch'], history_df['train_loss'], linewidth=2)
    axes[0].set_title('Total Loss', fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].grid(True, alpha=0.3)
    
    # Component losses
    axes[1].plot(history_df['epoch'], history_df['train_prediction'], label='Prediction', linewidth=2)
    axes[1].plot(history_df['epoch'], history_df['train_reconstruction'], label='Reconstruction', linewidth=2)
    axes[1].plot(history_df['epoch'], history_df['train_smoothness'], label='Smoothness', linewidth=2)
    axes[1].set_title('Loss Components', fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## 5. 可视化动态因子权重（核心创新！）

展示TFA学到的时变因子重要性


In [ ]:
# 创建分析器
analyzer = TFAAnalyzer(tfa, device=device)

# 提取因子权重
weights_df = analyzer.extract_factor_weights(X_train)

print(f"\\n🎯 Factor Weights Extracted:")
print(f"  Shape: {weights_df.shape}")
print(f"  Columns: {weights_df.columns.tolist()}")
print(f"\\nSample:")
weights_df.head(10)


In [ ]:
# 可视化平均attention pattern
analyzer.plot_average_attention_pattern(weights_df)

print("\\n💡 Interpretation:")
print("  - 热力图显示哪些PCA因子在哪些时期最重要")
print("  - 折线图显示attention如何随时间衰减")
print("  - 如果最近3个月权重高 → 动量策略")
print("  - 如果均匀分布 → 长期均值回复策略")


## 6. 分析Latent Factors

查看TFA学到的低维表示


In [ ]:
# 提取latent factors
latent_df, correlations = analyzer.analyze_latent_factors(X_train, y_train)

print(f"\\n🧬 Latent Factors:")
print(f"  Shape: {latent_df.shape}")
print(f"\\n  Correlations with Returns:")
for factor, corr in correlations.items():
    print(f"    {factor}: {corr:.4f}")

latent_df.head()


In [ ]:
# 可视化latent factors
analyzer.plot_latent_factor_analysis(latent_df, correlations)

print("\\n💡 Key Observations:")
print("  1. 左上：哪个latent factor与收益相关性最强")
print("  2. 右上：Latent factors之间的相关性（应该接近对角）")
print("  3. 左下：最佳factor与收益的散点图")
print("  4. 右下：各factor的分布")


## 7. 生成预测和交易信号


In [ ]:
# 生成预测
predictions = tfa.predict(X_train)

print(f"\\n📈 Predictions:")
print(f"  Shape: {predictions.shape}")
print(f"  Range: [{predictions.min():.4f}, {predictions.max():.4f}]")
print(f"  Mean: {predictions.mean():.4f}")

# 预测vs实际
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 散点图
axes[0].scatter(predictions, y_train, alpha=0.3, s=10)
axes[0].plot([predictions.min(), predictions.max()], 
             [predictions.min(), predictions.max()],
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Predicted Return')
axes[0].set_ylabel('Actual Return')
axes[0].set_title('Prediction vs Actual', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 相关性
corr = np.corrcoef(predictions, y_train)[0, 1]
axes[0].text(0.05, 0.95, f'Correlation: {corr:.3f}',
            transform=axes[0].transAxes,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 分档分析
pred_df = pd.DataFrame({'prediction': predictions, 'actual': y_train})
pred_df['decile'] = pd.qcut(pred_df['prediction'], 10, labels=False, duplicates='drop')
decile_returns = pred_df.groupby('decile')['actual'].mean()

axes[1].bar(decile_returns.index, decile_returns.values, alpha=0.7, color='steelblue')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Prediction Decile (0=Low, 9=High)')
axes[1].set_ylabel('Average Actual Return')
axes[1].set_title('Decile Analysis (Monotonicity Check)', fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\\n✅ Good model: Decile returns should increase monotonically")


## 总结：TFA的核心优势

### 🎯 创新点

1. **Dynamic Factor Weighting**
   - PCA因子权重不是固定的
   - 根据历史36个月动态调整
   - 可解释：看attention就知道哪些时期重要

2. **Encoder-Decoder架构**
   - 不只预测，还重构原始PCA因子
   - 确保学到的表示保留信息
   - 防止过拟合

3. **Temporal Smoothness**
   - 平滑性约束让权重变化更渐进
   - 增强可解释性
   - 符合金融直觉

### 📊 相比传统方法

| 维度 | 传统PCA | TFA |
|------|---------|-----|
| 因子权重 | 静态 | 时变 |
| 时序建模 | 无 | Transformer |
| 可解释性 | 载荷矩阵 | Attention权重 |
| 预测目标 | 无 | 有（end-to-end） |

### 🚀 下一步

1. 用完整数据训练：`python train_tfa.py --epochs 50`
2. 对比baseline：Ridge, LSTM等
3. Regime分析：牛熊市下的不同attention pattern
4. 写论文！

### 📚 论文核心Table

```
Table X: TFA vs Baselines

Model          | IC    | ICIR  | Sharpe
---------------|-------|-------|--------
PCA + Ridge    | 0.042 | 0.58  | 0.95
PCA + LSTM     | 0.052 | 0.71  | 1.28
TFA (Ours)     | 0.065 | 0.89  | 1.65   ← 目标
```
